<a href="https://colab.research.google.com/github/yingchengsun/AI-Job-Notes/blob/master/Data_process.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import json
from collections import OrderedDict
import urllib
from bs4 import BeautifulSoup
import pandas as pd
import binascii
import pickle
import base64
import re

yelp="https://www.yelp.com"

In [0]:
'''
transfer the original URL to 16 bit based coding address
'''
def transURL(s):
    #s="a b c"
    trans=''
    for x in s:
        code = ord(x)
        if (code>64 and code<91) or (code>96 and code<123) or (code>47 and code<58):
            trans += x
        else:
            #trans += "%" + x.encode("hex") #python 2.7
            x = x.encode('utf-8') #python 3
            trans += "%" + bytes.decode(base64.b16encode(x)) #python 3
    return trans

In [0]:
def download_images(reviewBody):
    pre_images=reviewBody.findAll('div',{"class":"photo-box photo-box--interactive is-loading"})
    for pre_image in pre_images:
        image = pre_image.find('img')
        image_url = image['data-async-src']
        image_name = image_url.split('/')[-2]
        urllib.urlretrieve(image_url,'pictures/'+'%s.jpg' % image_name)

        more =  reviewBody.find('li',{"class":"more-review-photos"})
        if more:
            print more.find('a')['href']

In [0]:
def extractReviews(url):
    start = 0
    num_pages = 1
    end = 20* num_pages

    #ratings =[]
    #reviews =[]
    rid = []
    totalPhotos = []
    rev_scraped = {}
    
    while (start < end):
        url_ = url + '?start=' + str(start)
        start +=20
        print (url_)
        page = urllib.request.urlopen(url_)
        soup = BeautifulSoup(page)

        r= soup.find("script",{"type":"application/json","data-hypernova-key":"yelp_main__6685fde41a25f43c936f7d1d7c8536b9cd17b120__yelp_main__BizDetailsApp__dynamic"})
        r_json = (r.text.lstrip('<!--').rstrip('-->'))
        data_item = json.loads(r_json, object_pairs_hook=OrderedDict)
        reviews = data_item['bizDetailsPageProps']['reviewFeedQueryProps']['reviews']
        for item in reviews:
          rid.append(item['id'])
          print (item['totalPhotos'])
          print (item['feedback']['voterText'])
        #for r in soup.findAll(string=re.compile("s2PtRuePlmtTalWaZ-_2dA")):
        #  print (r)
        '''
        for reviewBody in soup.findAll('div',{"itemprop":"review"}):
            print (reviewBody)
            ratings.append(reviewBody.div.meta['content'])
            reviews.append(reviewBody.find('p').text)
            #download_images(reviewBody)
    print (ratings)
    print (reviews)
    '''
url = "https://www.yelp.com/biz/emerald-chinese-restaurant-mississauga?osq=Emerald+Chinese+Restaurant?start=0"
extractReviews(url)

https://www.yelp.com/biz/emerald-chinese-restaurant-mississauga?osq=Emerald+Chinese+Restaurant?start=0?start=0
DafRBSrCIEoq0nCzITIoog
fP7fWf992wqkjlTcenfJcA
IU6yg6aJEPZc7D40uA3bFQ
aH6nrQeushCvkUqX2bqGmQ
mvjE-QM7chMmuaghi7Rr_Q
kOsOgmzXxZU6bjGkkjShHg
Dx2RkfKQGjbsf2glKSc5qw
Y-JKPycv5LcrL6DDRVjtLA
isTvyUp-dCrEUCdkCHUsjQ
Eqtyouaqn1gtTr8tMehF6A
Nb_2kqrkj64TnVaOedj2FA
jZ1qTNjwk-UXeEYNdjWzlA
qcFgnVnGBdfebzrZdbkeHw
Hm_UL7Y1zu4cVJehHXeDqw
VdSZBFLAmPMbPNpGhCb6nw
KE2EC7gIe7dUVAxx68V-MA
Iz8xMfZDWT4HS-3cKgJIFg
6W0MQHmasK0IsaoDo4bmkw
7PHMFeSTOuoKDXY0C4iaHw
yFbffgoW-CUq1eqabYlnmw


In [44]:
'''
Collect "recommendated" and "not recommended" reviews by business ID 
#Main entrance
'''

with open('/content/drive/My Drive/Yelp/yelp_dataset/business.json', 'r') as biz_inf:
  i= 0
  for idx, line in enumerate(biz_inf):
      print (idx, line)
      if idx >2:
        break
       
      '''
      read business from 'business.json'
      '''
      data_item = json.loads(line, object_pairs_hook=OrderedDict)
      biz_name = data_item['name']
      print (biz_name)
      biz_postal = data_item['postal_code']
      
      '''
      Search the business info and get the first one in the result list as the target
      '''
      search_url = yelp+"/search?find_desc="+transURL(biz_name)+"&find_loc="+transURL(biz_postal)
      #print (search_url)
      search_result = urllib.request.urlopen(search_url)   
      soup = BeautifulSoup(search_result)    
      rankings = soup.findAll("p",{"class":"lemon--p__373c0__3Qnnj text__373c0__2pB8f text-color--black-regular__373c0__38bRH text-align--left__373c0__2pnx_ text-size--inherit__373c0__2gFQ3"})
      for item in rankings:
        if item.text[0:2]=="1.": #the first one in the result list
          biz_url = yelp + p.a['href']
      print (biz_url)
      
      #extractReviews(biz_url)
      '''
      soup = BeautifulSoup(biz_url, 'html.parser')
      #link = soup.find_all("a", class_="lemon--a__373c0__IEZFH link__373c0__29943 link-color--inherit__373c0__15ymx link-size--default__373c0__1skgq")
      #link = soup.find_all(re.compile("not currently recommended"))
      NR = soup.find("a", string=re.compile("not currently recommended"))
      NR_URL = yelp+NR['href']
      NR_Number = NR.text.split()[0]
      print (NR_Number)
      extract_NR_Reviews(NR_URL)
      '''
                    

0 {"business_id":"1SWheh84yJXfytovILXOAQ","name":"Arizona Biltmore Golf Club","address":"2818 E Camino Acequia Drive","city":"Phoenix","state":"AZ","postal_code":"85016","latitude":33.5221425,"longitude":-112.0184807,"stars":3.0,"review_count":5,"is_open":0,"attributes":{"GoodForKids":"False"},"categories":"Golf, Active Life","hours":null}

Arizona Biltmore Golf Club
https://www.yelp.com/biz/the-cowfish-sushi-burger-bar-charlotte?osq=Musashi+Japanese+Restaurant
1 {"business_id":"QXAEGFB4oINsVuTFxEYKFQ","name":"Emerald Chinese Restaurant","address":"30 Eglinton Avenue W","city":"Mississauga","state":"ON","postal_code":"L5R 3E7","latitude":43.6054989743,"longitude":-79.652288909,"stars":2.5,"review_count":128,"is_open":1,"attributes":{"RestaurantsReservations":"True","GoodForMeal":"{'dessert': False, 'latenight': False, 'lunch': True, 'dinner': True, 'brunch': False, 'breakfast': False}","BusinessParking":"{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False

In [0]:
def extract_NR_Reviews(url):
    start = 0
    num_pages = 1
    end = 10* num_pages

    ratings =[]
    reviews =[]
    outfile = open("/content/drive/My Drive/Yelp/yelp_dataset/ReviewID.txt",'wb')
    rids=[]
    while (start < end):
        url_ = url + '?start=' + str(start)
        start +=20
        print (url_)
        page = urllib.request.urlopen(url_)
        soup = BeautifulSoup(page)

        for reviewBody in soup.findAll('div',{"class":"review review--with-sidebar"}):
            rids.append(reviewBody['data-review-id'])
            #print (reviewBody.find('p').text)
            #ratings.append(reviewBody.div.meta['content'])
            #reviews.append(reviewBody.find('p').text)
            #download_images(reviewBody)
    pickle.dump(rids,outfile)
    outfile.close()
    #print (ratings)
    #print (reviews)
    #review-wrapper

In [31]:
print (1==1)

True
